<a href="https://colab.research.google.com/github/plaban1981/Hugging_Face_transformers_topics/blob/main/Audio_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 5.8 MB 23.2 MB/s 
     |████████████████████████████████| 7.6 MB 52.9 MB/s 
     |████████████████████████████████| 182 kB 64.4 MB/s 


In [2]:
! pip install git+https://github.com/openai/whisper.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-2p6dku4n
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-2p6dku4n
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175257 sha256=900b31f4db87088f91535a81cb89d27f600cf4fbcc9591940656095d35054a12
  Stored in directory: /tmp/pip-ephem-wheel-cache-37ek7omo/wheels/a7/70/18/b7693c07b1d18b3dafb328f5d0496aa0d41a9c09ef332fd8e6
Successfully built whisper


In [3]:
import transformers
transformers.__version__

'4.25.1'

In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Download youtube video

In [15]:
! pip install youtube-dl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 27.6 MB/s 


In [16]:
from __future__ import unicode_literals
import youtube_dl


ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192',
    }],
    'outtmpl':"." + '/video.%(ext)s',
}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=srSNZlbhCwk&t=3s'])
    
absolute_path = "video.wav" #file name of your downloaded audio

[youtube] srSNZlbhCwk: Downloading webpage
[download] Destination: ./video.webm
[download] 100% of 763.32KiB in 00:17
[ffmpeg] Destination: ./video.wav
Deleting original file ./video.webm (pass -k to keep)


## Transcribe text from Audio

In [1]:
import whisper
model = whisper.load_model("large")
options = dict(language='en', beam_size=5, best_of=5)
#transcribe_options = dict(task="transcribe", **options)
translate_options = dict(task="translate", **options)
translation = model.transcribe("/content/video.wav", **translate_options)

In [3]:
translation

{'text': " This happened when I was 12 years old, I went to my grandmother's house with my mom My uncle had a computer at that time, which had a lot of games, so I started playing games as soon as I went there It was night and everyone went to attend the wedding in a nearby village except me Because I wanted to play games on the computer, then my uncle called me that his car has been punctured There is a tire in the house, we are close, come with the tire When I went to him, I saw that a strange thing was sitting, I asked him who are you Then he showed his face, which was very scary, then I noticed that he was just up to the stomach I ran immediately from there, but I fainted while running, when my eyes opened, I was at my grandmother's house I still don't know what it was",
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 5.4,
   'text': " This happened when I was 12 years old, I went to my grandmother's house with my mom",
   'tokens': [639,
    2011,
    562,
    286

In [4]:
translation['text']

" This happened when I was 12 years old, I went to my grandmother's house with my mom My uncle had a computer at that time, which had a lot of games, so I started playing games as soon as I went there It was night and everyone went to attend the wedding in a nearby village except me Because I wanted to play games on the computer, then my uncle called me that his car has been punctured There is a tire in the house, we are close, come with the tire When I went to him, I saw that a strange thing was sitting, I asked him who are you Then he showed his face, which was very scary, then I noticed that he was just up to the stomach I ran immediately from there, but I fainted while running, when my eyes opened, I was at my grandmother's house I still don't know what it was"

In [14]:
import joblib
joblib.dump(model,'whisper')

['whisper']

## Zeroshot classification using transformers

https://www.section.io/engineering-education/how-to-implement-zero-shot-classification-using-python/#:~:text=Zero-shot%20classification%20is%20a%20technique%20that%20allows%20us,topic%2C%20emotion%2C%20or%20event%20described%20by%20the%20label.

In [6]:
from transformers import pipeline
classifier_pipeline = pipeline ("zero-shot-classification", model = "facebook/bart-large-mnli")

In [7]:
label_candidate = ['Positive', 'Negative', 'Neutral']
classifier_pipeline (translation['text'], label_candidate)

{'sequence': " This happened when I was 12 years old, I went to my grandmother's house with my mom My uncle had a computer at that time, which had a lot of games, so I started playing games as soon as I went there It was night and everyone went to attend the wedding in a nearby village except me Because I wanted to play games on the computer, then my uncle called me that his car has been punctured There is a tire in the house, we are close, come with the tire When I went to him, I saw that a strange thing was sitting, I asked him who are you Then he showed his face, which was very scary, then I noticed that he was just up to the stomach I ran immediately from there, but I fainted while running, when my eyes opened, I was at my grandmother's house I still don't know what it was",
 'labels': ['Negative', 'Neutral', 'Positive'],
 'scores': [0.6785243153572083, 0.19358287751674652, 0.12789280712604523]}

## Yout